In [1]:
from utils import *
from data_block import *
from transforms import *
from transformers import GPT2Tokenizer
from model import *
from learner import *
from callbacks import *
# from xresnets import *
import torch

In [2]:
import matplotlib.pyplot as plt

In [3]:
tfms = [into_rgb, ResizeFixed(224), np_to_float]

In [4]:
path = Path('C:/Users/iamab/Downloads/datasets/coco/')
il = ImageList.from_path(path, tfms = tfms, include = ['train2014', 'val2014'])
sl = SplitDataset.from_ratio(il, ratio=0.05)

In [6]:
sl

SplitDataset
Train: ImageList (117123 items)
[WindowsPath('C:/Users/iamab/Downloads/datasets/coco/train2014/COCO_train2014_000000018873.jpg'), WindowsPath('C:/Users/iamab/Downloads/datasets/coco/val2014/COCO_val2014_000000151651.jpg'), WindowsPath('C:/Users/iamab/Downloads/datasets/coco/val2014/COCO_val2014_000000171382.jpg'), WindowsPath('C:/Users/iamab/Downloads/datasets/coco/train2014/COCO_train2014_000000478511.jpg'), WindowsPath('C:/Users/iamab/Downloads/datasets/coco/train2014/COCO_train2014_000000092017.jpg'), WindowsPath('C:/Users/iamab/Downloads/datasets/coco/train2014/COCO_train2014_000000299712.jpg'), WindowsPath('C:/Users/iamab/Downloads/datasets/coco/val2014/COCO_val2014_000000055074.jpg'), WindowsPath('C:/Users/iamab/Downloads/datasets/coco/train2014/COCO_train2014_000000524790.jpg'), WindowsPath('C:/Users/iamab/Downloads/datasets/coco/val2014/COCO_val2014_000000255708.jpg'), WindowsPath('C:/Users/iamab/Downloads/datasets/coco/val2014/COCO_val2014_000000170292.jpg')...]
V

In [5]:
def show_pic_and_caption(l, index):
    tok = l.py.tokenizer
    im = l.x[index]
    label = l.y.items[index]
    print(tok.decode(label.numpy(), skip_special_tokens = True))
    plt.imshow(im.permute(1,2,0))
    plt.axis('off')
    
# sh = partial(show_pic_and_caption, il)

In [6]:
def read_file(path):
    with open(path, encoding = 'utf-8') as f:
        ls = [i.strip() for i in f.readlines()]
        return ls
    
def conv_to_dict(file):
    dic = {}
    for i in file:
        j = i.split('\t')
        dic[j[0]] = j[1]
    return dic

def caption_label(o, label_dict,caption_number=1):
    return label_dict[o.name + f'#{caption_number}']

In [7]:
ll = label_train_valid(sl, 
                      func = partial(caption_label, label_dict = conv_to_dict(read_file(path/'ftd/token.txt'))),
                  procy = Tokprocessor(GPT2Tokenizer.from_pretrained('gpt2')))

I0716 08:56:26.379752  6424 tokenization_utils.py:1022] loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json from cache at C:\Users\iamab/.cache\torch\transformers\f2808208f9bec2320371a9f5f891c184ae0b674ef866b79c58177067d15732dd.1512018be4ba4e8726e41b9145129dc30651ea4fec86aa61f4b9f40bf94eac71
I0716 08:56:26.380744  6424 tokenization_utils.py:1022] loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt from cache at C:\Users\iamab/.cache\torch\transformers\d629f792e430b3c76a1291bb2766b0a047e36fae0588f9dbc1ae51decdff691b.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
E0716 08:56:26.587119  6424 tokenization_utils.py:548] Using pad_token, but it is not set yet.
I0716 08:56:26.589106  6424 tokenization_utils.py:1177] Adding <BOS> to the vocabulary
I0716 08:56:26.590106  6424 tokenization_utils.py:1255] Assigning <BOS> to the bos_token key of the tokenizer
I0716 08:56:26.591102  6424 tokenization_utils.py:1177] Adding <

In [8]:
def pad_collate(samples, pad_idx,flip = False):
    images = torch.stack([s[0] for s in samples])
    captions = [s[1].contiguous() for s in samples]
    if flip:
        for s in captions:
            idx = next(j for j,n in enumerate(s) if n == pad_idx)
            cut = s[:idx].flip(0)
            s[:idx] = cut
    res = torch.stack([s[1] for s in samples])
            
    return (torch.stack([s[0] for s in samples]), res.long()), torch.cat((res[:,1:], torch.zeros(res.size(0),1).int()+pad_idx), dim=1).long()

def cap_databunchify(sd, train_bs = None,valid_bs = None,**kwargs):
    return DataBunch(*get_cap_dls(sd, train_bs, valid_bs, collate_fn = pad_collate))

SplitDataset.cap_databunch = cap_databunchify

data = ll.cap_databunch(train_bs=8, num_workers = 2)

In [12]:
import pickle

In [15]:
torch.save(data, 'data.pkl')

In [75]:
x,y = next(iter(data.train_dl))

In [76]:
x[1][0].dtype

torch.int64

In [16]:
sh = partial(show_pic_and_caption,ll.train)

In [22]:
import torchvision.models as models
mod = models.resnet34(pretrained=True)

## Learning with pretrained visual backbone

In [23]:
model = get_caption_model(data, visual_head= mod, N = 4)

loss = FlatLoss(nn.CrossEntropyLoss())

cbfs = [CudaCapCallback, partial(AvgStatsCallback, cap_accuracy, AvgStatsCap), ProgressCallback, 
        normalise_callback(data, normalise_func= cap_normalise), NewRecorderCallback]

In [91]:
learn = Learner(model, data, loss, cb_funcs= cbfs)

In [92]:
learn.fit(3)

epoch,train_loss,train_cap_accuracy,valid_loss,valid_cap_accuracy,time
0,9.066910,0.682687,5.070878,0.706860,20:45
1,3.504230,0.725086,1.949681,0.787381,20:34
2,1.929187,0.786250,1.791563,0.802081,20:34


In [88]:
out = model((x[0].cuda(), x[1].cuda()))

## Learning from scratch

In [16]:
import torchvision.models as models
mod = models.resnet34(pretrained= False)

In [17]:
model = get_caption_model(data, visual_head= mod, N = 4)

loss = FlatLoss(nn.CrossEntropyLoss())

cbfs = [CudaCapCallback, partial(AvgStatsCallback, cap_accuracy, AvgStatsCap), ProgressCallback, 
        normalise_callback(data, normalise_func= cap_normalise), NewRecorderCallback]

In [18]:
learn = Learner(model, data, loss, cb_funcs= cbfs)

In [19]:
learn.fit(3)

epoch,train_loss,train_cap_accuracy,valid_loss,valid_cap_accuracy,time
0,8.520174,0.704436,4.396590,0.726308,20:45
1,3.222858,0.722765,2.855051,0.726308,20:39
2,2.888842,0.724005,2.769075,0.726308,20:35
